# Importovanje datoteka

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import json
import pandas as pd
import re
import html
import os
import shutil
import time
import random

# Prikupljanje i spašavanje linkova svih članaka svih kategorija


Funkcija za uzimanje svih linkova članaka sa jedne stranice neke kategorije

In [ ]:
def get_all_links_from_page(page_url):
  page=requests.get(page_url)
  soup=BeautifulSoup(page.text,'html.parser')

  links=set()
  for link in soup.find_all('a', class_="entry-featured-image-url", href=True):
    href=link['href']
    links.add(href)
  return list(links)

Funkcija za određivanje ukupnog broja pageva neke kategorije

In [ ]:
def get_num_pages(base_url, category_name):
    page_number = 1  # prva stranica
    max_page = 1  # broj pageva neke kategorije

    while True:
        url = f"{base_url}/{category_name}/page/{page_number}/?et_blog"
        response = requests.get(url)  # Pristup sledećoj stranici

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            next_page_button = soup.find('a', string='« Older Entries')  #trazenje dugmeta za pomjeranje na starije clanke tjst pageve

            if next_page_button:
                # Ako postoji dugme za nazad
                page_number += 1
                max_page = page_number
            else:
                # ako nema dugmeta tjt kraj pageva
                break
        else:
            print(f"Greška prilikom pristupa stranici: {response.status_code}")
            break

    return max_page

Funkcija za dohvacanje svih linkova članaka neke kategorije kroz sve stranice kategorije

In [ ]:
def get_all_links_from_category(base_url, category_name, max_pages):
  all_links=set()
  page_num=1

  while page_num<=max_pages:
    category_url = f"{base_url}/{category_name}/page/{page_num}/"

    links_from_page=get_all_links_from_page(category_url)
    if not links_from_page:
      break

    all_links.update(links_from_page)
    page_num=page_num+1
  return all_links

Prikupljanje linkova svih članaka svih kategorija koristeći ove gore funkcije

In [ ]:
base_url="https://www.pressrs.ba/"

categories=["financije","ljepota-i-zdravlje","ljubav-i-veze","sport","tech","casino","zanimljivosti","znanost"]
all_links=set()

for category in categories:
  max_pages=get_num_pages(base_url,category)
  all_category_links=get_all_links_from_category(base_url,category,max_pages)
  all_links.update(all_category_links)

  #spasavanje svih linkova od jedne kategorije u zasebnu datoteku
  with open(f'{category}_links.txt', 'w', encoding='utf-8') as f:
        for link in all_category_links:
            f.write(link + '\n')
  print(f"Broj članaka u kategoriji '{category}': {len(all_category_links)}")

print("Ukupan broj članaka sveukupno: ",len(all_links))

Broj članaka u kategoriji 'financije': 87
Broj članaka u kategoriji 'ljepota-i-zdravlje': 12
Broj članaka u kategoriji 'ljubav-i-veze': 75
Broj članaka u kategoriji 'sport': 10
Broj članaka u kategoriji 'tech': 15
Broj članaka u kategoriji 'casino': 99
Broj članaka u kategoriji 'zanimljivosti': 78
Broj članaka u kategoriji 'znanost': 10
Ukupan broj članaka sveukupno:  386


# Folder u kojem će se pohraniti txt datoteke svakog članka

In [ ]:
folder="VijestiPoKategorijama"
os.makedirs(folder,exist_ok=True)

all_data=[]
count=0
target_date=datetime.today()

# Scrapovanje podataka članaka iz svake kategorije

In [ ]:
for category in categories:
  links_file=f'{category}_links.txt'
  if os.path.exists(links_file):
    with open(links_file, 'r', encoding='utf-8') as linkovi:
      with open('SviClanci.txt','a', encoding='utf-8') as f:
        for i, line in enumerate(linkovi):
          url=line.strip()

          #mockovanje korisnika
          user={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
          response=requests.get(url,headers=user)

          if response.status_code!=200:
            print(f"Greška pri dohvaćanju stranice: {response.status_code}")
          else:
            content = response.content.decode('utf-8')
            content = content.replace("<br />", " ").replace("<br>", " ").replace(
              "<br style=\"box-sizing: border-box; margin: 0px; padding: 0px; border: none; outline: 0px\">", " ")
            soup = BeautifulSoup(content, 'html.parser')

            portal="pressrs.ba"

            #datum objave clanka
            meta_date = soup.find('meta', attrs={'property': 'article:published_time'})
            if meta_date and 'content' in meta_date.attrs:
              date_str = datetime.strptime(meta_date['content'][:10], '%Y-%m-%d').strftime('%d.%m.%Y')
            else:
              date_str = "N/A"

            #autor
            meta_author = soup.find('meta', attrs={'name': 'author'})
            author = meta_author['content'] if meta_author and 'content' in meta_author.attrs else "N/A"

            #naslov
            title = soup.title.string
            title=title.split(" - ")[0]

            #podnaslov
            div_content = soup.find_all('div', class_='et_pb_text_inner')
            podnaslovi = []
            for div in div_content:
              podnaslovi_div = div.find_all('h2')
              for podnaslov in podnaslovi_div:
                podnaslovi.append(podnaslov.get_text())

            podnaslovi_string = ", ".join(podnaslovi)

            #sadrzaj
            paragrafi = []
            for div in div_content:
              paragrafi_div = div.find_all('p')
              for paragraf in paragrafi_div:
              # da li paragraf sadrži h2 tag, ako ne, dodaj ga
                if not paragraf.find('h2'):
                  paragrafi.append(paragraf.get_text())

            paragrafi_string=" ".join(paragrafi)

            index_tagovi = paragrafi_string.find('Tagovi')

            # ako se "Tagovi" nalazi u stringu, skratimo sadržaj
            if index_tagovi != -1:
              paragrafi_string = paragrafi_string[:index_tagovi].strip()

            #strana
            meta_tag = soup.find('meta', property='og:url')
            if meta_tag:
              url_page = meta_tag['content']

            #upisivanje u fajl
            f.write("<***>\n")
            f.write(f"NOVINA: {portal}\n")
            f.write(f"DATUM: {date_str}\n")
            f.write(f"RUBRIKA: {category}\n")
            f.write(f"NADNASLOV: N/A\n")
            f.write(f"NASLOV: {title}\n")
            f.write(f"PODNASLOV: {podnaslovi_string}\n")
            f.write(f"STRANA: {url}\n")
            f.write(f"AUTOR(I): {author}\n\n")
            f.write(f"{paragrafi_string}\n\n")

            # Save to a separate file for each article
            sanitized_title = re.sub(r'[\\/*?:"<>|\t]', "", title)
            sanitized_title = re.sub(r'\s+', " ", sanitized_title).strip()
            filepath = f"{sanitized_title}.txt"

            if not sanitized_title:
             sanitized_title = f"clanak_{i}"

            filepath = os.path.join(folder, f"{sanitized_title}_{i}.txt")

            #filepath = os.path.join(folder, filepath)
            with open(filepath, 'w', encoding='utf-8') as article_file:
                article_file.write("<***>\n")
                article_file.write(f"NOVINA: {portal}\n")
                article_file.write(f"DATUM: {date_str}\n")
                article_file.write(f"RUBRIKA: {category}\n")
                article_file.write(f"NADNASLOV: N/A\n")
                article_file.write(f"NASLOV: {title}\n")
                article_file.write(f"PODNASLOV: {podnaslovi_string}\n")
                article_file.write(f"STRANA: {url}\n")
                article_file.write(f"AUTOR(I): {author}\n\n")
                article_file.write(f"{paragrafi_string}\n\n")
  else:
    print(f"File {links_file} does not exist.")

In [ ]:
broj_fajlova = len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

print(f"U direktoriju '{folder}' ima {broj_fajlova} fajlova.")

U direktoriju 'VijestiPoKategorijama' ima 386 fajlova.


In [ ]:
file_path = 'SviClanci.txt'
podstring = 'AUTOR(I): Drazen'

with open(file_path, 'r', encoding='utf-8') as f:
    sadrzaj = f.read()

broj_ponavljanja = sadrzaj.count(podstring)

print(f"Podstring '{podstring}' se pojavljuje {broj_ponavljanja} puta u fajlu.")

Podstring 'AUTOR(I): Drazen' se pojavljuje 386 puta u fajlu.


# Spašavanje txt fajla na drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Putanja do fajla u Colabu
source_path = '/content/SviClanci.txt'

# Lokacija na Google Driveu gdje želiš da ga spremiš
destination_path = '/content/drive/My Drive/Kazazovic_Amina'

# Kopiranje fajla
import shutil
shutil.copy(source_path, destination_path)

'/content/drive/My Drive/Kazazovic_Amina/SviClanci.txt'

# TOKENIZACIJA

prolazak kroz svaki tekst clanka i vrsenje tokenizacije

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

def prebroji_tokene_po_clanku(putanja_do_fajla):
    sav_tekst = ""
    with open(putanja_do_fajla, 'r', encoding='utf-8') as f:
        sav_tekst = f.read()

    clanci = sav_tekst.split("<***>")
    ukupno_svih_tokena = 0

    for i, clanak in enumerate(clanci):
        linije = clanak.strip().split('\n')
        pocetak_sadrzaja = -1
        broj_tokena_clanka = 0
        if i==0:
          continue

        for j, linija in enumerate(linije):
            if linija.startswith("AUTOR(I):"):
                if j + 2 < len(linije):
                    pocetak_sadrzaja = j + 2
                    break

        if pocetak_sadrzaja != -1:
            sadrzaj_clanka = "\n".join(linije[pocetak_sadrzaja:]).strip()
            if sadrzaj_clanka:
                tokenizer = Tokenizer()
                tokenizer.fit_on_texts([sadrzaj_clanka])
                tokens = tokenizer.texts_to_sequences([sadrzaj_clanka])
                broj_tokena_clanka = sum(len(sekvenca) for sekvenca in tokens)
                ukupno_svih_tokena += broj_tokena_clanka
                print(f"Broj tokena za članak {i}: {broj_tokena_clanka}")

    print(f"\nUkupan broj tokena u sadržaju svih članaka: {ukupno_svih_tokena}")

putanja_do_svih_pdfova = "/content/drive/My Drive/Kazazovic_Amina/SviClanci.txt"

# Prebrojite tokene po članku
prebroji_tokene_po_clanku(putanja_do_svih_pdfova)

Broj tokena za članak 1: 669
Broj tokena za članak 2: 864
Broj tokena za članak 3: 869
Broj tokena za članak 4: 1106
Broj tokena za članak 5: 760
Broj tokena za članak 6: 1062
Broj tokena za članak 7: 464
Broj tokena za članak 8: 1074
Broj tokena za članak 9: 990
Broj tokena za članak 10: 1062
Broj tokena za članak 11: 476
Broj tokena za članak 12: 812
Broj tokena za članak 13: 1060
Broj tokena za članak 14: 863
Broj tokena za članak 15: 1060
Broj tokena za članak 16: 999
Broj tokena za članak 17: 997
Broj tokena za članak 18: 467
Broj tokena za članak 19: 595
Broj tokena za članak 20: 1048
Broj tokena za članak 21: 967
Broj tokena za članak 22: 921
Broj tokena za članak 23: 874
Broj tokena za članak 24: 749
Broj tokena za članak 25: 458
Broj tokena za članak 26: 1137
Broj tokena za članak 27: 598
Broj tokena za članak 28: 985
Broj tokena za članak 29: 464
Broj tokena za članak 30: 1095
Broj tokena za članak 31: 841
Broj tokena za članak 32: 723
Broj tokena za članak 33: 1036
Broj toke